In [1]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import AdamW

from torchvision import datasets, transforms

from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

import os
import random
from tqdm import tqdm
from PIL import Image

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(7)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
import torchvision.models as models
model = models.efficientnet_v2_s(pretrained=True)

num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)

model = nn.DataParallel(model, device_ids=[0, 1])
model = model.to(device)
#모델 업로드 경로 지정
model_path = '/kaggle/input/model4444/pytorch/model4/1/model4.pth'
model.load_state_dict(torch.load(model_path))
model.eval()
final_predictions = []

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 152MB/s] 


In [4]:
target_size = 380

test_transform = transforms.Compose([
    transforms.Resize(target_size, interpolation=transforms.InterpolationMode.BILINEAR),  # 양선형 보간법 사용
    transforms.CenterCrop(target_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_name = self.df.iloc[idx, 0]
        image_path = f'/kaggle/input/image-classification-2024-spring/dataset/test/{image_name}'
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        return image

print(target_size)

test_df = pd.read_csv('/kaggle/input/image-classification-2024-spring/submission_sample.csv')

test_dataset = CustomDataset(test_df, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

with torch.no_grad():
    for images in tqdm(test_dataloader):
        images = images.to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs)
        _, predicted = torch.max(preds.data, 1)
        final_predictions += predicted.detach().cpu().numpy().tolist()

test_df['label'] = final_predictions

test_df.to_csv('submission_model4.csv', index=False)

380


100%|██████████| 32/32 [00:51<00:00,  1.62s/it]
